In [ ]:
%load_ext autoreload
%autoreload true

In [ ]:
import sys
sys.path.insert(0, '../..')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score

from utils import DataManager
from Substation_real import *

# Instantiation of the models

In [ ]:
from sklearn.decomposition import PCA

scaler_knn = PCA(n_components=4, whiten=True)

from sklearn.neighbors import KNeighborsClassifier

model_knn = KNeighborsClassifier(algorithm='ball_tree', leaf_size=25, metric='euclidean',
                     n_jobs=20, n_neighbors=3, p=1.2478426560306763)

In [ ]:
from sklearn.decomposition import PCA

scaler_lr = PCA(n_components=4, whiten=True)

from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression(C=0.3699214191568069, l1_ratio=0.5, max_iter=709, n_jobs=20,
                   random_state=1, solver='sag', tol=0.002743766415124)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler_rf = MinMaxScaler(clip=True, feature_range=(-1.0, 1.0))

from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(bootstrap=False, class_weight='balanced',
                       criterion='entropy', max_features=0.12077862686737162,
                       min_samples_leaf=2, n_estimators=82, n_jobs=20,
                       random_state=0, verbose=False)

In [ ]:
from sklearn.decomposition import PCA

scaler_svc = PCA(n_components=5, whiten=True)

from sklearn.svm import LinearSVC

model_svc = LinearSVC(C=1.4154375206825454, intercept_scaling=1.1127529228889177,
          max_iter=1311, multi_class='crammer_singer', random_state=3,
          tol=0.0003672100970153386)

In [ ]:
from sklearn.decomposition import PCA

scaler_xgb = PCA(n_components=5, whiten=True)

from xgboost import XGBClassifier

model_xgb = XGBClassifier(base_score=0.5, booster='gbtree',
              colsample_bylevel=0.9180722541805479, colsample_bynode=1,
              colsample_bytree=0.9109072964153705, enable_categorical=False,
              gamma=0.005048898721121109, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.44104052517602826,
              max_delta_step=0, max_depth=5, min_child_weight=5, missing=np.nan,
              monotone_constraints='()', n_estimators=2600, n_jobs=20,
              num_parallel_tree=1, predictor='auto', random_state=4,
              reg_alpha=0.869216551449954, reg_lambda=1.9967334510937407,
              scale_pos_weight=1, seed=4, subsample=0.6654978224569946,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

## Using LR in that case

In [ ]:
scaler = scaler_lr
model = model_lr

# Loading train data and training model

In [ ]:
from load_fault_dataset import SubstationFaultLoader

loader = SubstationFaultLoader(os.path.join(DATA_ROOT, 'Substation-20221015'))

train_faults = pd.read_csv(TRAIN_SPEC, index_col=0)
data_train, metadata_train = loader.load_fault_dataset_df(list(train_faults.index.values))
dm_train = DataManager()
dm_train.prepare_database(data_train, metadata_train, del_cols=HIDDEN_VARS+['anomaly'])

X_train, y_train = dm_train.split_X_Y_concat()
X_train['TIN_s_K'] = 48 + 237.15
X_train['TOUT_s_K'] = 70 + 237.15

In [ ]:
scaler.fit(X_train)

In [ ]:
model.fit(scaler.transform(X_train), y_train)

# Load test data and computing predictions

In [ ]:
df_raw = pd.read_csv('real/podstanica_vreme.csv', header=0, index_col=1, parse_dates=True)
df_test = pd.DataFrame()
df_test['TIN_p_K'] = df_raw['tnp'] + 273.15
df_test['TOUT_p_K'] = df_raw['tpp'] + 273.15
df_test['TIN_s_K'] = df_raw['tps'] + 273.15
df_test['TOUT_s_K'] = df_raw['tns'] + 273.15
df_test['DemandPower_kW'] = df_raw['qizm']

df_test.dropna(inplace=True)

X_test = df_test

In [ ]:
y_pred = model.predict(scaler.transform(X_test))
y_test = 0 * y_pred # assuming no fault

# Computing model scores

In [ ]:
mcc = matthews_corrcoef(y_test,	y_pred)
print("MCC: %.3f" % mcc)

acc = accuracy_score(y_test, y_pred)
print("Accuracy: %.3f" % acc)

In [ ]:
# knn = {'scaler': scaler, 'model': model}
# save_object(knn, 'model_knn.pickle')

# Plotting sample results

In [ ]:
from Yacine.model_evaluation_plots import *

y_prob = model.predict_proba(scaler.transform(X_test))
y_pred = alert_trigger(horizon=1, proba=y_prob[:,1], threshold=0.5)

acc = accuracy_score(y_test, y_pred)
print("Accuracy: %.3f" % acc)

In [ ]:
x_DTLM = (X_test['TIN_p_K'] - X_test['TOUT_s_K']) - (X_test['TOUT_p_K'] - X_test['TIN_s_K']) \
    / np.log((X_test['TIN_p_K'] - X_test['TOUT_s_K'])/(X_test['TOUT_p_K']-X_test['TIN_s_K']))
UA = X_test['DemandPower_kW'] / x_DTLM

UA_mean = UA.fillna(method='pad').rolling(24).mean()
UA_mean[UA_mean<0] = 0
UA_mean[UA_mean>7] = 7

In [ ]:
fault_var_data = pd.DataFrame(1000 * UA_mean, columns=['var'])

df = pd.DataFrame({'true': y_test, 'pred' : y_pred})
df['t_pred'] = df.loc[df['true'] == df['pred'], 'pred']
df['f_pred'] = df.loc[df['true'] != df['pred'], 'pred']
df.index = fault_var_data.index

df_plot = pd.merge(df, fault_var_data, left_index=True, right_index=True)

In [ ]:
save_object(df_plot, 'resu_sst_lr.pickle')

In [ ]:
def plot_true_false_2(ax, df, legend=False, twin_legend=False):
    ax.plot(df.index, df['true'] , label='Ground truth', color='blue')
    ax.plot(df.index, df['t_pred'], 'x', label='Correct Fault Detection', color='green')
    ax.plot(df.index, df['f_pred'], 'x', label='Wrong Fault Detection', color='red')

    ax.set_ylabel('Classification (0: no fault, 1: fault)')
    ax.set_xlabel('Date') 

    if legend:
        ax.legend()

    axtwin1 = ax.twinx()
    axtwin1.plot(df.index, df['var'], label=fault_var_label, color='grey')

    axtwin1.set_ylabel('Estimated UA [W/K]')

    if twin_legend:
        axtwin1.legend()

In [ ]:
df_plot1 = df_plot.loc['2017-02-01':'2017-04-30']
df_plot2 = df_plot.loc['2018-09-01':'2019-04-30']
df_plot3 = df_plot.loc['2019-09-01':'2020-04-30']

fig, ax = plt.subplots(3, 1, figsize=(25,15))
plot_true_false_2(ax[0], df_plot1, twin_legend=True)

plot_true_false_2(ax[1], df_plot2)
plot_true_false_2(ax[2], df_plot3, legend=True)